# COGS 181 Neural Networks & Deep Learning Final Project

## Setup

### Import the necessary packages

In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import math
import random

from PIL import Image

from sklearn.preprocessing import OneHotEncoder

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torchvision.io import read_image, ImageReadMode

In [56]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', device)

def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

Device: cuda:0


### Configuration

In [190]:
# General Configuration

batch_size = 8
num_epochs = 15
image_size = 64

# GAN Configuration

dataroot = 'data/data_gan'
nc = 3
nz = 128
ngf = 72
ndf = 72
beta1 = 0.7
ngpu = 1
lr_gan = 0.0004

# Random seed
manualSeed = 360
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed);

Random Seed:  360


### Image Transformations

In [215]:
transform_seq = transforms.Compose([
    transforms.Resize(image_size, antialias=True),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

## Deep Convolutional Generative Adversarial Network

### Loading DCGAN Data

In [216]:
dataset_gan = dset.ImageFolder(root=dataroot,transform=transform_seq)

# Create the dataloader
dataloader_gan = torch.utils.data.DataLoader(dataset_gan, batch_size=batch_size, shuffle=True)

### DCGAN Setup

In [217]:
# initialize special weight creation for both discriminator and generator

def weights_init(w):
    label = w.__class__.__name__
    
    if label.find('Conv') != -1:
        nn.init.normal_(w.weight.data, 0.0, 0.02)
        
    elif label.find('BatchNorm') != -1:
        nn.init.normal_(w.weight.data, 1.0, 0.02)
        nn.init.constant_(w.bias.data, 0)

In [218]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        
        self.ngpu = ngpu
        
        self.main = nn.Sequential(
            
            nn.ConvTranspose2d(nz, ngf * 8, 4, 2, 0, bias=True),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=True),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=True),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=True),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=True),
            nn.Tanh()
            
        )
    
    def forward(self, input):
        
        return self.main(input)

In [220]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
    
    def forward(self, input):
        return self.main(input)

In [221]:
netG = Generator(ngpu).to(device)

netG.apply(weights_init);

netD = Discriminator(ngpu).to(device)

netD.apply(weights_init);

In [222]:
criterion = nn.BCELoss()

fixed_noise = torch.randn(64, nz, 1, 1, device=device)

real_label = 1
fake_label = 0

optimizerD = optim.Adam(netD.parameters(), lr=lr_gan, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr_gan, betas=(beta1, 0.999))

In [223]:
netD.to(device)
netG.to(device);

In [224]:
img_list = []
G_losses = []
D_losses = []
iters = 0

print('Starting training loop...')
for epoch in range(num_epochs):
    
    for i, data in enumerate(dataloader_gan, 0):
        
        # this first set of samples will be real data
        netD.zero_grad()
        # send data to GPU
        real_cpu = data[0].to(device)
        
        b_size = real_cpu.size(0)
        
        label = torch.full((b_size,), real_label, dtype=torch.float64, device=device)
        # predict on sample with discriminator
        output = netD(real_cpu).view(-1)
        
        # calculate the loss for this sample
        errD_real = criterion(output, label)
        
        # backpropogate the loss through the network
        errD_real.backward()
        
        
        # second set of samples will be fake or generated data
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        # generate fake set of data
        fake = netG(noise)

        label.fill_(fake_label)
        
        # attempt to predict fake data with discriminator
        output = netD(fake.detach()).view(-1)
        
        # calculate the loss of discrimninator
        errD_fake = criterion(output, label)
        # backpropogate the loss to discriminator
        errD_fake.backward()
        # update the weights
        optimizerD.step()
        
        # now update the generator with loss
        netG.zero_grad()
        
        label.fill_(real_label)
        
        output = netD(fake).view(-1)
        
        # calculate loss so far
        errG = criterion(output, label)
        
        # backpropogate loss
        errG.backward()
        
        # update weights
        optimizerG.step()
        
        # output every 100 batches in order to see progress
        if i % 100 == 0:
            print('Working...')
        iters += 1

Starting training loop...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...


## References

[[1] Optimal ratio for data splitting](https://onlinelibrary.wiley.com/doi/full/10.1002/sam.11583)

[[2] DCGAN Tutorial - PyTorch](https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html)

[[3] Implementing Generative Adversarial Networks (GANs) for Increasing a Convolutional Neural Network’s (CNN) Performance](https://12ft.io/proxy?q=https%3A%2F%2Ftowardsdatascience.com%2Fimplementing-generative-adversarial-networks-gans-for-increasing-a-convolutional-neural-networks-f871e17fe271)

[[3] Convolutional Neural Networks](https://12ft.io/proxy?q=https%3A%2F%2Fmedium.com%2Fswlh%2Ftraining-deep-neural-networks-on-a-gpu-with-pytorch-11079d89805)

https://towardsdatascience.com/complete-guide-to-adam-optimization-1e5f29532c3d

https://www.kdnuggets.com/2022/12/tuning-adam-optimizer-parameters-pytorch.html

## Data

[Fashionpedia](https://fashionpedia.github.io/home/Fashionpedia_download.html)

[Clothing Dataset - Kaggle](https://www.kaggle.com/datasets/agrigorev/clothing-dataset-full)